In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyMuPDFLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [12]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [21]:
loader = TextLoader("history.txt")
docs = loader.load()

In [25]:
file_path = 'korean_civil_act.pdf'
loader = PyMuPDFLoader(file_path)
docs = loader.load()

In [26]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, length_function=len)
splits = text_splitter.split_documents(docs)

In [27]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [28]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [29]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [39]:
q = "혼인 신고와 관련된 내용 알려줘. 답변시, 관련된 법조항 번호도 꼭 알려줘"

In [40]:
rag_chain.invoke(q)

'혼인은 가족관계의 등록 등에 관한 법률에 의해 혼인신고를 통해 성립됩니다. 혼인신고는 당사자 쌍방과 성년자인 증인 2인의 연서한 서면으로 이루어져야 합니다. 혼인의 신고는 관련 법령을 위반하지 않는 경우에만 수리됩니다. (혼인의 성립: 민법 제812조)'

In [41]:
llm.invoke(q)

AIMessage(content='혼인 신고란 결혼을 한 사실을 법적으로 공식적으로 등록하는 절차를 말합니다. 대한민국에서는 혼인신고는 주민등록법에 근거하여 이루어지며, 주민등록법 제5조에 따라 혼인신고를 하여야 합니다. \n\n주민등록법 제5조에 따르면 혼인 신고는 혼인을 한 후 3개월 이내에 하여야 하며, 혼인신고는 주민센터나 구청에 신고서를 제출하여야 합니다. 또한, 주민등록법 제7조에 따르면 혼인신고서에는 다음과 같은 내용이 포함되어야 합니다.\n1. 신고자, 배우자, 부모의 성명, 주소\n2. 혼인신고일자\n3. 혼인신고장소 등\n\n혼인신고를 하게 되면, 주민등록부에 신고된 내용이 등재되어 법적으로 혼인 관계가 인정받을 수 있습니다. 따라서 혼인을 한 경우에는 반드시 혼인신고를 해야 합니다.', response_metadata={'token_usage': {'completion_tokens': 341, 'prompt_tokens': 54, 'total_tokens': 395}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-60c02978-22b1-4bac-8464-f72a1ee75ac0-0')